# Import data

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

import os

DATA_DIR = os.path.join(
    path,
    "asl_alphabet_train",
    "asl_alphabet_train"
)


c:\Users\diepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\diepa\.cache\kagglehub\datasets\grassknoted\asl-alphabet\versions\1


# Import lib

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


c:\Users\diepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# Chuan hoa du lieu

In [3]:
IMG_SIZE = 64
BATCH_SIZE = 128

In [4]:
# cai thien datagen
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)


In [5]:
train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # chi dinh lay tap train
)

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # chi dinh lay tap validation
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


# Khai bao mo hinh - fit

In [6]:
# Add BatchNormalization and stronger regularization
from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # increase dropout
    Dense(29)  # no activation here, use from_logits=True
])

model.summary()

c:\Users\diepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,468,829 (9.42 MB)

 Trainable params: 2,468,381 (9.42 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# huan luyen
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

Epoch 1/10
544/544 ━━━━━━━━━━━━━━━━━━━━ 774s 1s/step - accuracy: 0.3335 - loss: 2.5457 - val_accuracy: 0.2328 - val_loss: 2.8963
Epoch 2/10
 61/544 ━━━━━━━━━━━━━━━━━━━━ 10:08 1s/step - accuracy: 0.5175 - loss: 1.9893

# Save model

In [ ]:
model.save('asl_alphabet_model.h5') # .h5: dung de load lai va du doan cho sau nay
print('Saved model')

Saved model


# Danh gia mo hinh

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {loss:.4f}')

136/136 ━━━━━━━━━━━━━━━━━━━━ 84s 615ms/step - accuracy: 0.7702 - loss: 0.9140
Accuracy: 0.7702
Loss: 0.9140


# Du doan

In [ ]:
# load mo hinh 
loaded_model = keras.models.load_model('asl_alphabet_model.h5')
print('Loaded model from disk')

Loaded model from disk


In [ ]:
# chuan bi label (chu cai)
class_names = list(train_data.class_indices.keys())
print(class_names)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [ ]:
# load 1 hinh de xu ly
img_dir = os.path.join(path, "asl_alphabet_train", "asl_alphabet_train", "M")  # duong dan den thu muc chua hinh
# Get first image file
img_file = os.listdir(img_dir)[0]
img_path = os.path.join(img_dir, img_file) # duong dan den hinh can du doan
# img_path = input("Enter the path of the image to predict: ")
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0)  # them kich thuoc

In [ ]:
# du doan
prediction = loaded_model.predict(img_array)
predicted_index = np.argmax(prediction)
predicted_label = class_names[predicted_index] # map index -> label

# print % cac lop du doan
print(prediction)
print(f'Predicted label: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
[[  7.2047033  -6.8893337 -14.328305  -23.54764     4.05158   -15.014789
   -2.3398588  -7.9824944  -5.204087    3.5110974 -32.4075    -11.335097
   20.111029   18.32067    -2.1878278 -17.007864  -13.633353  -21.710262
   -1.4844259  -9.353946  -26.852173  -37.27363   -28.499361  -12.969586
  -22.960167  -17.89233    -8.111152  -28.774622  -23.921034 ]]
Predicted label: M
